In [1]:
import pandas as pd
import openpyxl

In [5]:
def clean_penalties(path):
    stats_df = pd.read_excel(path, sheet_name="Penalties", engine="openpyxl")
    stats_df = stats_df.drop([0], axis = "index")
    stats_df = stats_df.rename(columns=stats_df.iloc[0]).drop(stats_df.index[0])
    stats_df.columns = ["HOME_skaternum_1",
                    "HOME_1_1",
                    "HOME_2_1",
                    "HOME_3_1",
                    "HOME_4_1",
                    "HOME_5_1",
                    "HOME_6_1",
                    "HOME_7_1",
                    "HOME_8_1",
                    "HOME_9_1",
                    "not_needed", 
                    "not_needed",
                    "not_needed",
                    "not_needed",
                    "not_needed",
                    "AWAY_skaternum_1",
                    "AWAY_1_1",
                    "AWAY_2_1",
                    "AWAY_3_1",
                    "AWAY_4_1",
                    "AWAY_5_1",
                    "AWAY_6_1",
                    "AWAY_7_1",
                    "AWAY_8_1",
                    "AWAY_9_1",
                    "not_needed",
                    "not_needed",
                    "not_needed",
                    "HOME_skaternum_2",
                    "HOME_1_2",
                    "HOME_2_2",
                    "HOME_3_2",
                    "HOME_4_2",
                    "HOME_5_2",
                    "HOME_6_2",
                    "HOME_7_2",
                    "HOME_8_2",
                    "HOME_9_2",
                    "not_needed", 
                    "not_needed",
                    "not_needed",
                    "not_needed",
                    "not_needed",
                    "AWAY_skaternum_2",
                    "AWAY_1_2",
                    "AWAY_2_2",
                    "AWAY_3_2",
                    "AWAY_4_2",
                    "AWAY_5_2",
                    "AWAY_6_2",
                    "AWAY_7_2",
                    "AWAY_8_2",
                    "AWAY_9_2",
                    "not_needed",
                    "not_needed",
                    "not_needed"]
    stats_df = stats_df.reset_index(drop=True)
    stats_df = stats_df.drop(["not_needed"], axis = 1)
    stats_df = stats_df.fillna("Zero")
    away_period_2 = stats_df.filter([
                    "AWAY_skaternum_2",
                    "AWAY_1_2",
                    "AWAY_2_2",
                    "AWAY_3_2",
                    "AWAY_4_2",
                    "AWAY_5_2",
                    "AWAY_6_2",
                    "AWAY_7_2",
                    "AWAY_8_2",
                    "AWAY_9_2"], axis=1)
    away_period_2["period"] = 2
    away_period_2["team"] = "away"
    away_period_1 = stats_df.filter([
                    "AWAY_skaternum_1",
                    "AWAY_1_1",
                    "AWAY_2_1",
                    "AWAY_3_1",
                    "AWAY_4_1",
                    "AWAY_5_1",
                    "AWAY_6_1",
                    "AWAY_7_1",
                    "AWAY_8_1",
                    "AWAY_9_1"], axis=1)
    away_period_1["period"] = 1
    away_period_1["team"] = "away"
    home_period_2 = stats_df.filter([
                    "HOME_skaternum_2",
                    "HOME_1_2",
                    "HOME_2_2",
                    "HOME_3_2",
                    "HOME_4_2",
                    "HOME_5_2",
                    "HOME_6_2",
                    "HOME_7_2",
                    "HOME_8_2",
                    "HOME_9_2"], axis=1)
    home_period_2["period"] = 2
    home_period_2["team"] = "home"
    home_period_1 = stats_df.filter([
                    "HOME_skaternum_1",
                    "HOME_1_1",
                    "HOME_2_1",
                    "HOME_3_1",
                    "HOME_4_1",
                    "HOME_5_1",
                    "HOME_6_1",
                    "HOME_7_1",
                    "HOME_8_1",
                    "HOME_9_1"], axis=1)
    home_period_1["period"] = 1
    home_period_1["team"] = "home"
    
    home_period_1 = clean_penalty_dfs(home_period_1)    
    home_period_2 = clean_penalty_dfs(home_period_2)   
    away_period_1 = clean_penalty_dfs(away_period_1)   
    away_period_2 = clean_penalty_dfs(away_period_2)
    
    frames = [home_period_1, home_period_2, away_period_1, away_period_2]
    result_df = pd.concat(frames).reset_index(drop=True)
    
    return result_df

In [10]:
def clean_penalty_dfs(df):
    df.columns = ["skater_number",
                        "column_1",
                        "column_2",
                        "column_3",
                        "column_4",
                        "column_5",
                        "column_6",
                        "column_7",
                        "column_8",
                        "column_9",
                        "period",
                        "team"]     
    for index, row in df.iterrows():
        if row.skater_number != "Zero":
            number = row.skater_number
        else:
            df.iloc[index, 0] = number
    df = df.set_index(df.skater_number, drop=True)
    df = df.drop(columns="skater_number") 
    series_func = df.column_1.\
                    append(df.column_2).\
                    append(df.column_3).\
                    append(df.column_4).\
                    append(df.column_5).\
                    append(df.column_6).\
                    append(df.column_7).\
                    append(df.column_8).\
                    append(df.column_9)   
    df = pd.DataFrame({"jam_number": series_func, 
                        "penalty_code": series_func,
                        "period": df["period"][0],
                        "team": df["team"][0]})
    
    df = df.drop("Non-Skater Expulsions").reset_index()
    for index, row in df.iterrows(): 
        if isinstance(row.jam_number, str):
            df.iloc[index, 1] = None       
    for index, row in df.iterrows():
        if isinstance(row.penalty_code, str):
            code = row.penalty_code
        else: 
            df.iloc[index, 2] = code              
    df = df.dropna().reset_index(drop=True)
    return df